# Oxygen XML Author Add-on for AI Text Markup

This add-on integrates with your local Flask API to provide AI-powered text markup functionality directly within Oxygen XML Author.

## Features
- Select text in XML document and apply AI markup
- Configure local Flask API endpoint and authentication
- Choose from available Ollama models
- Preview changes before applying
- Direct replacement or copy results

## Development Structure
- **Main Plugin**: Core functionality and action registration
- **Configuration**: Settings dialog for API configuration
- **UI Components**: Preview and replacement dialogs

In [ ]:
// filepath: c:\Project\OCR\Oxygen-add-on.ipynb
// Main Plugin Extension Class

package com.oxygenxml.ai.markup;

import java.awt.event.ActionEvent;
import java.net.URL;
import javax.swing.AbstractAction;
import javax.swing.JMenuItem;

import ro.sync.exml.plugin.PluginDescriptor;
import ro.sync.exml.plugin.Plugin;
import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;
import ro.sync.exml.workspace.api.standalone.MenuBarCustomizer;
import ro.sync.exml.workspace.api.standalone.ui.Menu;

/**
 * Main plugin class for AI Text Markup add-on
 */
public class AITextMarkupPlugin extends Plugin {
    
    /**
     * Plugin descriptor
     */
    public static final PluginDescriptor DESCRIPTOR = new PluginDescriptor() {
        @Override
        public String getName() {
            return "AI Text Markup";
        }

        @Override
        public String getDescription() {
            return "AI-powered text markup using local Ollama models via Flask API";
        }

        @Override
        public String getVersion() {
            return "1.0.0";
        }

        @Override
        public String getVendor() {
            return "Custom Development";
        }
    };

    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    @Override
    public void applicationStarted(StandalonePluginWorkspace pluginWorkspaceAccess) {
        this.pluginWorkspace = pluginWorkspaceAccess;
        this.configManager = new AIConfigurationManager(pluginWorkspaceAccess);
        
        // Add menu items
        addMenuItems();
    }

    private void addMenuItems() {
        pluginWorkspace.addMenuBarCustomizer(new MenuBarCustomizer() {
            @Override
            public void customizeMainMenu(MenuBarCustomizer menuBar) {
                // Create AI menu
                Menu aiMenu = new Menu("AI Markup");
                
                // Add markup action
                JMenuItem markupItem = new JMenuItem(new AIMarkupAction(
                    "Markup Selected Text", pluginWorkspace, configManager));
                aiMenu.add(markupItem);
                
                // Add separator
                aiMenu.addSeparator();
                
                // Add configuration action
                JMenuItem configItem = new JMenuItem(new AIConfigurationAction(
                    "Configure AI Service", pluginWorkspace, configManager));
                aiMenu.add(configItem);
                
                // Add to menu bar
                menuBar.addMenu(aiMenu);
            }
        });
    }

    @Override
    public boolean applicationClosing() {
        // Save configuration on exit
        if (configManager != null) {
            configManager.saveConfiguration();
        }
        return true;
    }
}

/**
 * Action for AI text markup
 */
class AIMarkupAction extends AbstractAction {
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    public AIMarkupAction(String name, StandalonePluginWorkspace workspace, 
                         AIConfigurationManager configManager) {
        super(name);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        // Get selected text from current editor
        String selectedText = getSelectedText();
        
        if (selectedText == null || selectedText.trim().isEmpty()) {
            pluginWorkspace.showInformationMessage("Please select text to markup.");
            return;
        }

        // Check if configuration is valid
        if (!configManager.isConfigured()) {
            pluginWorkspace.showInformationMessage(
                "Please configure AI service first (AI Markup → Configure AI Service).");
            return;
        }

        // Show markup dialog
        AIMarkupDialog dialog = new AIMarkupDialog(
            pluginWorkspace, configManager, selectedText);
        dialog.setVisible(true);
    }

    private String getSelectedText() {
        try {
            WSEditor currentEditor = pluginWorkspace.getCurrentEditorAccess(
                StandalonePluginWorkspace.MAIN_EDITING_AREA);
            
            if (currentEditor != null) {
                AuthorAccess authorAccess = currentEditor.getAuthorAccess();
                if (authorAccess != null) {
                    AuthorDocumentController controller = 
                        authorAccess.getDocumentController();
                    
                    int selectionStart = authorAccess.getEditorAccess()
                        .getSelectionStart();
                    int selectionEnd = authorAccess.getEditorAccess()
                        .getSelectionEnd();
                    
                    if (selectionStart != selectionEnd) {
                        return controller.getChars(selectionStart, 
                            selectionEnd - selectionStart);
                    }
                }
            }
        } catch (Exception ex) {
            ex.printStackTrace();
        }
        return null;
    }
}

/**
 * Action for AI configuration
 */
class AIConfigurationAction extends AbstractAction {
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    public AIConfigurationAction(String name, StandalonePluginWorkspace workspace,
                               AIConfigurationManager configManager) {
        super(name);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        AIConfigurationDialog dialog = new AIConfigurationDialog(
            pluginWorkspace, configManager);
        dialog.setVisible(true);
    }
}

In [ ]:
// Configuration Manager Class

package com.oxygenxml.ai.markup;

import java.util.List;
import java.util.ArrayList;
import java.io.IOException;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.net.URI;
import java.time.Duration;

import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;
import ro.sync.exml.workspace.api.options.WSOptionsStorage;

import com.google.gson.Gson;
import com.google.gson.JsonObject;
import com.google.gson.JsonArray;

/**
 * Manages AI service configuration and API communication
 */
public class AIConfigurationManager {
    
    private static final String CONFIG_KEY_ENDPOINT = "ai.markup.endpoint";
    private static final String CONFIG_KEY_API_KEY = "ai.markup.apikey";
    private static final String CONFIG_KEY_MODEL = "ai.markup.model";
    private static final String CONFIG_KEY_TIMEOUT = "ai.markup.timeout";
    
    private static final String DEFAULT_ENDPOINT = "http://127.0.0.1:5000/ai";
    private static final int DEFAULT_TIMEOUT = 120;
    
    private StandalonePluginWorkspace pluginWorkspace;
    private WSOptionsStorage optionsStorage;
    private HttpClient httpClient;
    private Gson gson;
    
    public AIConfigurationManager(StandalonePluginWorkspace workspace) {
        this.pluginWorkspace = workspace;
        this.optionsStorage = workspace.getOptionsStorage();
        this.httpClient = HttpClient.newBuilder()
            .connectTimeout(Duration.ofSeconds(10))
            .build();
        this.gson = new Gson();
        
        // Load existing configuration
        loadConfiguration();
    }
    
    // Configuration properties
    private String endpoint = DEFAULT_ENDPOINT;
    private String apiKey = "";
    private String selectedModel = "";
    private int timeout = DEFAULT_TIMEOUT;
    
    public void loadConfiguration() {
        endpoint = optionsStorage.getOption(CONFIG_KEY_ENDPOINT, DEFAULT_ENDPOINT);
        apiKey = optionsStorage.getOption(CONFIG_KEY_API_KEY, "");
        selectedModel = optionsStorage.getOption(CONFIG_KEY_MODEL, "");
        timeout = Integer.parseInt(optionsStorage.getOption(CONFIG_KEY_TIMEOUT, 
            String.valueOf(DEFAULT_TIMEOUT)));
    }
    
    public void saveConfiguration() {
        optionsStorage.setOption(CONFIG_KEY_ENDPOINT, endpoint);
        optionsStorage.setOption(CONFIG_KEY_API_KEY, apiKey);
        optionsStorage.setOption(CONFIG_KEY_MODEL, selectedModel);
        optionsStorage.setOption(CONFIG_KEY_TIMEOUT, String.valueOf(timeout));
    }
    
    public boolean isConfigured() {
        return !endpoint.trim().isEmpty() && !apiKey.trim().isEmpty();
    }
    
    // Getters and setters
    public String getEndpoint() { return endpoint; }
    public void setEndpoint(String endpoint) { this.endpoint = endpoint; }
    
    public String getApiKey() { return apiKey; }
    public void setApiKey(String apiKey) { this.apiKey = apiKey; }
    
    public String getSelectedModel() { return selectedModel; }
    public void setSelectedModel(String model) { this.selectedModel = model; }
    
    public int getTimeout() { return timeout; }
    public void setTimeout(int timeout) { this.timeout = timeout; }
    
    /**
     * Test API connection and get available models
     */
    public List<String> getAvailableModels() throws Exception {
        String modelsUrl = endpoint + "/models";
        
        HttpRequest request = HttpRequest.newBuilder()
            .uri(URI.create(modelsUrl))
            .header("Authorization", "Bearer " + apiKey)
            .header("Content-Type", "application/json")
            .timeout(Duration.ofSeconds(10))
            .GET()
            .build();
        
        HttpResponse<String> response = httpClient.send(request, 
            HttpResponse.BodyHandlers.ofString());
        
        if (response.statusCode() != 200) {
            throw new Exception("Failed to get models: " + response.body());
        }
        
        // Parse response
        JsonObject jsonResponse = gson.fromJson(response.body(), JsonObject.class);
        JsonArray modelsArray = jsonResponse.getAsJsonArray("data");
        
        List<String> models = new ArrayList<>();
        for (int i = 0; i < modelsArray.size(); i++) {
            JsonObject model = modelsArray.get(i).getAsJsonObject();
            models.add(model.get("id").getAsString());
        }
        
        return models;
    }
    
    /**
     * Test API connection
     */
    public boolean testConnection() {
        try {
            getAvailableModels();
            return true;
        } catch (Exception e) {
            return false;
        }
    }
    
    /**
     * Send markup request to AI service
     */
    public String requestMarkup(String text, String prompt) throws Exception {
        String completionsUrl = endpoint + "/chat/completions";
        
        // Prepare request body
        JsonObject requestBody = new JsonObject();
        requestBody.addProperty("model", selectedModel);
        requestBody.addProperty("max_tokens", 1500);
        requestBody.addProperty("temperature", 0.7);
        
        JsonArray messages = new JsonArray();
        
        // System message with prompt
        JsonObject systemMessage = new JsonObject();
        systemMessage.addProperty("role", "system");
        systemMessage.addProperty("content", prompt);
        messages.add(systemMessage);
        
        // User message with text to markup
        JsonObject userMessage = new JsonObject();
        userMessage.addProperty("role", "user");
        userMessage.addProperty("content", text);
        messages.add(userMessage);
        
        requestBody.add("messages", messages);
        
        HttpRequest request = HttpRequest.newBuilder()
            .uri(URI.create(completionsUrl))
            .header("Authorization", "Bearer " + apiKey)
            .header("Content-Type", "application/json")
            .timeout(Duration.ofSeconds(timeout))
            .POST(HttpRequest.BodyPublishers.ofString(gson.toJson(requestBody)))
            .build();
        
        HttpResponse<String> response = httpClient.send(request, 
            HttpResponse.BodyHandlers.ofString());
        
        if (response.statusCode() != 200) {
            throw new Exception("AI request failed: " + response.body());
        }
        
        // Parse response
        JsonObject jsonResponse = gson.fromJson(response.body(), JsonObject.class);
        JsonArray choices = jsonResponse.getAsJsonArray("choices");
        
        if (choices.size() > 0) {
            JsonObject firstChoice = choices.get(0).getAsJsonObject();
            JsonObject message = firstChoice.getAsJsonObject("message");
            return message.get("content").getAsString();
        }
        
        throw new Exception("No response from AI service");
    }
}

In [ ]:
// UI Dialog Classes

package com.oxygenxml.ai.markup;

import java.awt.*;
import java.awt.event.*;
import java.util.List;
import javax.swing.*;
import javax.swing.border.EmptyBorder;

import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;

/**
 * Configuration dialog for AI service settings
 */
public class AIConfigurationDialog extends JDialog {
    
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;
    
    private JTextField endpointField;
    private JPasswordField apiKeyField;
    private JComboBox<String> modelComboBox;
    private JSpinner timeoutSpinner;
    private JButton testButton;
    private JLabel statusLabel;
    
    public AIConfigurationDialog(StandalonePluginWorkspace workspace, 
                               AIConfigurationManager configManager) {
        super((Frame) workspace.getParentFrame(), "AI Service Configuration", true);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
        
        initComponents();
        loadSettings();
        setupEventHandlers();
        
        setDefaultCloseOperation(DISPOSE_ON_CLOSE);
        pack();
        setLocationRelativeTo(workspace.getParentFrame());
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        
        // Main panel
        JPanel mainPanel = new JPanel(new GridBagLayout());
        mainPanel.setBorder(new EmptyBorder(15, 15, 15, 15));
        
        GridBagConstraints gbc = new GridBagConstraints();
        gbc.insets = new Insets(5, 5, 5, 5);
        gbc.anchor = GridBagConstraints.WEST;
        
        // Endpoint
        gbc.gridx = 0; gbc.gridy = 0;
        mainPanel.add(new JLabel("API Endpoint:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        endpointField = new JTextField(30);
        mainPanel.add(endpointField, gbc);
        
        // API Key
        gbc.gridx = 0; gbc.gridy = 1; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("API Key:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        apiKeyField = new JPasswordField(30);
        mainPanel.add(apiKeyField, gbc);
        
        // Model
        gbc.gridx = 0; gbc.gridy = 2; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("Model:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        modelComboBox = new JComboBox<>();
        modelComboBox.setEditable(true);
        mainPanel.add(modelComboBox, gbc);
        
        // Timeout
        gbc.gridx = 0; gbc.gridy = 3; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("Timeout (seconds):"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        timeoutSpinner = new JSpinner(new SpinnerNumberModel(120, 10, 600, 10));
        mainPanel.add(timeoutSpinner, gbc);
        
        // Test button
        gbc.gridx = 0; gbc.gridy = 4; gbc.gridwidth = 2; gbc.anchor = GridBagConstraints.CENTER;
        testButton = new JButton("Test Connection & Load Models");
        mainPanel.add(testButton, gbc);
        
        // Status label
        gbc.gridx = 0; gbc.gridy = 5; gbc.gridwidth = 2;
        statusLabel = new JLabel(" ");
        statusLabel.setHorizontalAlignment(SwingConstants.CENTER);
        mainPanel.add(statusLabel, gbc);
        
        add(mainPanel, BorderLayout.CENTER);
        
        // Button panel
        JPanel buttonPanel = new JPanel(new FlowLayout());
        JButton okButton = new JButton("OK");
        JButton cancelButton = new JButton("Cancel");
        
        buttonPanel.add(okButton);
        buttonPanel.add(cancelButton);
        add(buttonPanel, BorderLayout.SOUTH);
        
        // Button event handlers
        okButton.addActionListener(e -> saveAndClose());
        cancelButton.addActionListener(e -> dispose());
    }
    
    private void loadSettings() {
        endpointField.setText(configManager.getEndpoint());
        apiKeyField.setText(configManager.getApiKey());
        timeoutSpinner.setValue(configManager.getTimeout());
        
        // Try to load models if configuration exists
        if (configManager.isConfigured()) {
            loadModels();
        }
    }
    
    private void setupEventHandlers() {
        testButton.addActionListener(e -> testConnectionAndLoadModels());
    }
    
    private void testConnectionAndLoadModels() {
        // Update config with current values
        configManager.setEndpoint(endpointField.getText().trim());
        configManager.setApiKey(new String(apiKeyField.getPassword()));
        configManager.setTimeout((Integer) timeoutSpinner.getValue());
        
        testButton.setEnabled(false);
        statusLabel.setText("Testing connection...");
        statusLabel.setForeground(Color.BLUE);
        
        // Run in background thread
        SwingWorker<Void, Void> worker = new SwingWorker<Void, Void>() {
            private List<String> models;
            private Exception error;
            
            @Override
            protected Void doInBackground() throws Exception {
                try {
                    models = configManager.getAvailableModels();
                } catch (Exception e) {
                    error = e;
                }
                return null;
            }
            
            @Override
            protected void done() {
                testButton.setEnabled(true);
                
                if (error == null && models != null) {
                    // Success
                    statusLabel.setText("✓ Connection successful! Models loaded.");
                    statusLabel.setForeground(Color.GREEN);
                    
                    // Update model combo box
                    modelComboBox.removeAllItems();
                    for (String model : models) {
                        modelComboBox.addItem(model);
                    }
                    
                    // Select previously selected model if available
                    String selectedModel = configManager.getSelectedModel();
                    if (!selectedModel.isEmpty() && models.contains(selectedModel)) {
                        modelComboBox.setSelectedItem(selectedModel);
                    }
                    
                } else {
                    // Error
                    statusLabel.setText("✗ Connection failed: " + 
                        (error != null ? error.getMessage() : "Unknown error"));
                    statusLabel.setForeground(Color.RED);
                }
            }
        };
        
        worker.execute();
    }
    
    private void loadModels() {
        SwingWorker<Void, Void> worker = new SwingWorker<Void, Void>() {
            private List<String> models;
            
            @Override
            protected Void doInBackground() throws Exception {
                try {
                    models = configManager.getAvailableModels();
                } catch (Exception e) {
                    // Ignore errors when loading initial models
                }
                return null;
            }
            
            @Override
            protected void done() {
                if (models != null) {
                    modelComboBox.removeAllItems();
                    for (String model : models) {
                        modelComboBox.addItem(model);
                    }
                    
                    String selectedModel = configManager.getSelectedModel();
                    if (!selectedModel.isEmpty() && models.contains(selectedModel)) {
                        modelComboBox.setSelectedItem(selectedModel);
                    }
                }
            }
        };
        
        worker.execute();
    }
    
    private void saveAndClose() {
        // Validate inputs
        String endpoint = endpointField.getText().trim();
        String apiKey = new String(apiKeyField.getPassword());
        
        if (endpoint.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter an API endpoint.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        if (apiKey.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter an API key.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        // Save configuration
        configManager.setEndpoint(endpoint);
        configManager.setApiKey(apiKey);
        configManager.setTimeout((Integer) timeoutSpinner.getValue());
        
        Object selectedModel = modelComboBox.getSelectedItem();
        if (selectedModel != null) {
            configManager.setSelectedModel(selectedModel.toString());
        }
        
        configManager.saveConfiguration();
        
        dispose();
    }
}

/**
 * Main markup dialog for text processing
 */
public class AIMarkupDialog extends JDialog {
    
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;
    private String originalText;
    
    private JTextArea promptArea;
    private JTextArea originalTextArea;
    private JTextArea resultTextArea;
    private JButton processButton;
    private JButton replaceButton;
    private JButton copyButton;
    private JButton previewButton;
    private JLabel statusLabel;
    
    public AIMarkupDialog(StandalonePluginWorkspace workspace, 
                         AIConfigurationManager configManager, String selectedText) {
        super((Frame) workspace.getParentFrame(), "AI Text Markup", true);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
        this.originalText = selectedText;
        
        initComponents();
        setupEventHandlers();
        
        setDefaultCloseOperation(DISPOSE_ON_CLOSE);
        pack();
        setLocationRelativeTo(workspace.getParentFrame());
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        setSize(800, 600);
        
        // Create main panel with split panes
        JSplitPane mainSplit = new JSplitPane(JSplitPane.VERTICAL_SPLIT);
        
        // Top panel for prompt and controls
        JPanel topPanel = new JPanel(new BorderLayout());
        topPanel.setBorder(BorderFactory.createTitledBorder("AI Prompt"));
        
        promptArea = new JTextArea(4, 50);
        promptArea.setText("Please markup the following text with appropriate XML tags. " +
            "Follow the existing document structure and use standard markup patterns.");
        promptArea.setLineWrap(true);
        promptArea.setWrapStyleWord(true);
        topPanel.add(new JScrollPane(promptArea), BorderLayout.CENTER);
        
        // Control panel
        JPanel controlPanel = new JPanel(new FlowLayout());
        processButton = new JButton("Process with AI");
        controlPanel.add(processButton);
        
        statusLabel = new JLabel("Model: " + configManager.getSelectedModel());
        controlPanel.add(statusLabel);
        
        topPanel.add(controlPanel, BorderLayout.SOUTH);
        
        // Bottom panel for text comparison
        JSplitPane textSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        
        // Original text panel
        JPanel originalPanel = new JPanel(new BorderLayout());
        originalPanel.setBorder(BorderFactory.createTitledBorder("Original Text"));
        originalTextArea = new JTextArea(15, 30);
        originalTextArea.setText(originalText);
        originalTextArea.setEditable(false);
        originalTextArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        originalPanel.add(new JScrollPane(originalTextArea), BorderLayout.CENTER);
        
        // Result text panel
        JPanel resultPanel = new JPanel(new BorderLayout());
        resultPanel.setBorder(BorderFactory.createTitledBorder("AI Result"));
        resultTextArea = new JTextArea(15, 30);
        resultTextArea.setEditable(true);
        resultTextArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        resultPanel.add(new JScrollPane(resultTextArea), BorderLayout.CENTER);
        
        // Button panel for results
        JPanel resultButtonPanel = new JPanel(new FlowLayout());
        replaceButton = new JButton("Replace Original");
        copyButton = new JButton("Copy Result");
        previewButton = new JButton("Preview Diff");
        
        replaceButton.setEnabled(false);
        copyButton.setEnabled(false);
        previewButton.setEnabled(false);
        
        resultButtonPanel.add(replaceButton);
        resultButtonPanel.add(copyButton);
        resultButtonPanel.add(previewButton);
        resultPanel.add(resultButtonPanel, BorderLayout.SOUTH);
        
        textSplit.setLeftComponent(originalPanel);
        textSplit.setRightComponent(resultPanel);
        textSplit.setDividerLocation(400);
        
        mainSplit.setTopComponent(topPanel);
        mainSplit.setBottomComponent(textSplit);
        mainSplit.setDividerLocation(150);
        
        add(mainSplit, BorderLayout.CENTER);
        
        // Bottom button panel
        JPanel bottomButtonPanel = new JPanel(new FlowLayout());
        JButton closeButton = new JButton("Close");
        closeButton.addActionListener(e -> dispose());
        bottomButtonPanel.add(closeButton);
        
        add(bottomButtonPanel, BorderLayout.SOUTH);
    }
    
    private void setupEventHandlers() {
        processButton.addActionListener(e -> processWithAI());
        replaceButton.addActionListener(e -> replaceText());
        copyButton.addActionListener(e -> copyResult());
        previewButton.addActionListener(e -> showPreview());
    }
    
    private void processWithAI() {
        String prompt = promptArea.getText().trim();
        if (prompt.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter a prompt.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        processButton.setEnabled(false);
        statusLabel.setText("Processing with AI...");
        statusLabel.setForeground(Color.BLUE);
        resultTextArea.setText("");
        
        SwingWorker<String, Void> worker = new SwingWorker<String, Void>() {
            private Exception error;
            
            @Override
            protected String doInBackground() throws Exception {
                try {
                    return configManager.requestMarkup(originalText, prompt);
                } catch (Exception e) {
                    error = e;
                    return null;
                }
            }
            
            @Override
            protected void done() {
                processButton.setEnabled(true);
                
                try {
                    String result = get();
                    if (result != null) {
                        resultTextArea.setText(result);
                        statusLabel.setText("✓ Processing completed");
                        statusLabel.setForeground(Color.GREEN);
                        
                        replaceButton.setEnabled(true);
                        copyButton.setEnabled(true);
                        previewButton.setEnabled(true);
                    } else {
                        statusLabel.setText("✗ Processing failed: " + 
                            (error != null ? error.getMessage() : "Unknown error"));
                        statusLabel.setForeground(Color.RED);
                    }
                } catch (Exception e) {
                    statusLabel.setText("✗ Processing failed: " + e.getMessage());
                    statusLabel.setForeground(Color.RED);
                }
            }
        };
        
        worker.execute();
    }
    
    private void replaceText() {
        String result = resultTextArea.getText();
        if (result.trim().isEmpty()) {
            return;
        }
        
        int choice = JOptionPane.showConfirmDialog(this, 
            "Replace the selected text in the document with the AI result?", 
            "Confirm Replacement", JOptionPane.YES_NO_OPTION);
        
        if (choice == JOptionPane.YES_OPTION) {
            try {
                replaceSelectedText(result);
                JOptionPane.showMessageDialog(this, "Text replaced successfully!");
                dispose();
            } catch (Exception e) {
                JOptionPane.showMessageDialog(this, 
                    "Error replacing text: " + e.getMessage(), 
                    "Error", JOptionPane.ERROR_MESSAGE);
            }
        }
    }
    
    private void copyResult() {
        String result = resultTextArea.getText();
        if (!result.trim().isEmpty()) {
            Toolkit.getDefaultToolkit().getSystemClipboard()
                .setContents(new StringSelection(result), null);
            JOptionPane.showMessageDialog(this, "Result copied to clipboard!");
        }
    }
    
    private void showPreview() {
        String original = originalTextArea.getText();
        String result = resultTextArea.getText();
        
        if (result.trim().isEmpty()) {
            return;
        }
        
        // Create preview dialog
        JDialog previewDialog = new JDialog(this, "Preview Changes", true);
        previewDialog.setLayout(new BorderLayout());
        previewDialog.setSize(900, 500);
        
        // Create diff view (simple side-by-side comparison)
        JSplitPane diffSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        
        JPanel beforePanel = new JPanel(new BorderLayout());
        beforePanel.setBorder(BorderFactory.createTitledBorder("Before (Original)"));
        JTextArea beforeArea = new JTextArea(original);
        beforeArea.setEditable(false);
        beforeArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 11));
        beforePanel.add(new JScrollPane(beforeArea), BorderLayout.CENTER);
        
        JPanel afterPanel = new JPanel(new BorderLayout());
        afterPanel.setBorder(BorderFactory.createTitledBorder("After (AI Result)"));
        JTextArea afterArea = new JTextArea(result);
        afterArea.setEditable(false);
        afterArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 11));
        afterPanel.add(new JScrollPane(afterArea), BorderLayout.CENTER);
        
        diffSplit.setLeftComponent(beforePanel);
        diffSplit.setRightComponent(afterPanel);
        diffSplit.setDividerLocation(450);
        
        previewDialog.add(diffSplit, BorderLayout.CENTER);
        
        JPanel previewButtonPanel = new JPanel(new FlowLayout());
        JButton acceptButton = new JButton("Accept & Replace");
        JButton cancelButton = new JButton("Cancel");
        
        acceptButton.addActionListener(e -> {
            previewDialog.dispose();
            replaceText();
        });
        cancelButton.addActionListener(e -> previewDialog.dispose());
        
        previewButtonPanel.add(acceptButton);
        previewButtonPanel.add(cancelButton);
        previewDialog.add(previewButtonPanel, BorderLayout.SOUTH);
        
        previewDialog.setLocationRelativeTo(this);
        previewDialog.setVisible(true);
    }
    
    private void replaceSelectedText(String replacement) throws Exception {
        WSEditor currentEditor = pluginWorkspace.getCurrentEditorAccess(
            StandalonePluginWorkspace.MAIN_EDITING_AREA);
        
        if (currentEditor != null) {
            AuthorAccess authorAccess = currentEditor.getAuthorAccess();
            if (authorAccess != null) {
                AuthorDocumentController controller = 
                    authorAccess.getDocumentController();
                
                int selectionStart = authorAccess.getEditorAccess().getSelectionStart();
                int selectionEnd = authorAccess.getEditorAccess().getSelectionEnd();
                
                if (selectionStart != selectionEnd) {
                    controller.delete(selectionStart, selectionEnd - selectionStart);
                    controller.insertText(selectionStart, replacement);
                }
            }
        }
    }
}

In [ ]:
<!-- filepath: c:\Project\OCR\Oxygen-add-on.ipynb -->
<!-- Plugin Configuration Files -->

<!-- plugin.xml - Main plugin descriptor -->
<?xml version="1.0" encoding="UTF-8"?>
<plugin
    name="AI Text Markup"
    description="AI-powered text markup using local Ollama models via Flask API"
    version="1.0.0"
    vendor="Custom Development"
    class="com.oxygenxml.ai.markup.AITextMarkupPlugin">
    
    <runtime>
        <library name="lib/ai-markup-plugin.jar"/>
        <library name="lib/gson-2.8.9.jar"/>
    </runtime>
    
    <extension point="WorkspaceAccess" class="com.oxygenxml.ai.markup.AITextMarkupPlugin"/>
</plugin>

<!-- addon.xml - Add-on descriptor for packaging -->
<?xml version="1.0" encoding="UTF-8"?>
<xt:extensions xmlns:xt="http://www.oxygenxml.com/ns/extension"
              xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
              xsi:schemaLocation="http://www.oxygenxml.com/ns/extension 
                                 http://www.oxygenxml.com/ns/extension/extensions.xsd">
    
    <xt:extension id="ai.text.markup">
        <xt:location href="ai-markup-plugin.jar"/>
        <xt:version>1.0.0</xt:version>
        <xt:oxy_version>20.0+</xt:oxy_version>
        <xt:type>plugin</xt:type>
        <xt:author>Custom Development</xt:author>
        <xt:name>AI Text Markup</xt:name>
        <xt:description>
            AI-powered text markup using local Ollama models via Flask API.
            Select text in XML documents and apply intelligent markup using local AI models.
        </xt:description>
        <xt:license>
            <![CDATA[
            Custom License - For personal and educational use.
            ]]>
        </xt:license>
    </xt:extension>
</xt:extensions>

<!-- build.xml - Ant build script for compilation -->
<?xml version="1.0" encoding="UTF-8"?>
<project name="ai-markup-plugin" default="jar" basedir=".">
    
    <property name="src.dir" value="src"/>
    <property name="build.dir" value="build"/>
    <property name="lib.dir" value="lib"/>
    <property name="oxygen.dir" value="C:/Program Files/Oxygen XML Author 27"/>
    
    <path id="classpath">
        <fileset dir="${oxygen.dir}/lib" includes="*.jar"/>
        <fileset dir="${lib.dir}" includes="*.jar"/>
    </path>
    
    <target name="init">
        <mkdir dir="${build.dir}"/>
    </target>
    
    <target name="compile" depends="init">
        <javac srcdir="${src.dir}" 
               destdir="${build.dir}" 
               classpathref="classpath"
               includeantruntime="false"/>
    </target>
    
    <target name="jar" depends="compile">
        <jar destfile="${lib.dir}/ai-markup-plugin.jar" basedir="${build.dir}">
            <manifest>
                <attribute name="Plugin-Class" value="com.oxygenxml.ai.markup.AITextMarkupPlugin"/>
                <attribute name="Plugin-Name" value="AI Text Markup"/>
                <attribute name="Plugin-Version" value="1.0.0"/>
            </manifest>
        </jar>
    </target>
    
    <target name="clean">
        <delete dir="${build.dir}"/>
    </target>
    
    <target name="addon" depends="jar">
        <zip destfile="ai-markup-addon.zip">
            <fileset dir="." includes="plugin.xml,addon.xml"/>
            <fileset dir="${lib.dir}" includes="*.jar"/>
        </zip>
    </target>
</project>

## Installation Instructions

### 1. Prerequisites
- Download required dependencies:
  - Gson library: `gson-2.8.9.jar`
  - Place in `lib/` directory

### 2. Build the Plugin
```bash
# Compile and create JAR
ant jar

# Create add-on package
ant addon
```

### 3. Install in Oxygen XML Author
1. Open Oxygen XML Author
2. Go to **Help → Install new add-ons**
3. Click **Browse** and select `ai-markup-addon.zip`
4. Follow installation wizard
5. Restart Oxygen XML Author

### 4. Configure AI Service
1. Go to **AI Markup → Configure AI Service**
2. Enter your Flask API details:
   - **API Endpoint**: `http://127.0.0.1:5000/ai`
   - **API Key**: From your Flask API key manager
   - **Timeout**: 120 seconds (adjust as needed)
3. Click **Test Connection & Load Models**
4. Select desired model from dropdown
5. Click **OK** to save

### 5. Usage
1. Open an XML document in Oxygen
2. Select text you want to markup
3. Go to **AI Markup → Markup Selected Text**
4. Enter or modify the AI prompt
5. Click **Process with AI**
6. Review results and choose:
   - **Replace Original**: Replace selected text in document
   - **Copy Result**: Copy to clipboard
   - **Preview Diff**: Side-by-side comparison before replacing

## Features
- ✅ Integration with local Flask API
- ✅ Model selection from Ollama
- ✅ Configurable prompts
- ✅ Preview before replacement
- ✅ Side-by-side comparison
- ✅ Copy to clipboard functionality
- ✅ Persistent configuration
- ✅ Error handling and validation

## Configuration Files
- Settings stored in Oxygen's options storage
- Automatically persisted between sessions
- Test connection validates API accessibility
- Dynamic model loading from API

I'll create an Oxygen XML Author add-on that integrates with your local Flask API for AI-powered text markup. This will be implemented as three notebooks that work together.

## Step-by-step Solution:

1. **Create the main add-on implementation** - Core Java classes for the Oxygen plugin
2. **Implement the UI components** - Dialog for configuration, preview, and replacement
3. **Add configuration management** - Settings for API endpoint, keys, and model selection

### [Oxygen-add-on.ipynb](file:///c%3A/Project/OCR/Oxygen-add-on.ipynb)

Add complete implementation for the Oxygen XML Author add-on with AI integration.



In [ ]:
<VSCode.Cell language="markdown">
# Oxygen XML Author Add-on for AI Text Markup

This add-on integrates with your local Flask API to provide AI-powered text markup functionality directly within Oxygen XML Author.

## Features
- Select text in XML document and apply AI markup
- Configure local Flask API endpoint and authentication
- Choose from available Ollama models
- Preview changes before applying
- Direct replacement or copy results

## Development Structure
- **Main Plugin**: Core functionality and action registration
- **Configuration**: Settings dialog for API configuration
- **UI Components**: Preview and replacement dialogs
</VSCode.Cell>
<VSCode.Cell language="java">
// filepath: c:\Project\OCR\Oxygen-add-on.ipynb
// Main Plugin Extension Class

package com.oxygenxml.ai.markup;

import java.awt.event.ActionEvent;
import java.net.URL;
import javax.swing.AbstractAction;
import javax.swing.JMenuItem;

import ro.sync.exml.plugin.PluginDescriptor;
import ro.sync.exml.plugin.Plugin;
import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;
import ro.sync.exml.workspace.api.standalone.MenuBarCustomizer;
import ro.sync.exml.workspace.api.standalone.ui.Menu;

/**
 * Main plugin class for AI Text Markup add-on
 */
public class AITextMarkupPlugin extends Plugin {
    
    /**
     * Plugin descriptor
     */
    public static final PluginDescriptor DESCRIPTOR = new PluginDescriptor() {
        @Override
        public String getName() {
            return "AI Text Markup";
        }

        @Override
        public String getDescription() {
            return "AI-powered text markup using local Ollama models via Flask API";
        }

        @Override
        public String getVersion() {
            return "1.0.0";
        }

        @Override
        public String getVendor() {
            return "Custom Development";
        }
    };

    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    @Override
    public void applicationStarted(StandalonePluginWorkspace pluginWorkspaceAccess) {
        this.pluginWorkspace = pluginWorkspaceAccess;
        this.configManager = new AIConfigurationManager(pluginWorkspaceAccess);
        
        // Add menu items
        addMenuItems();
    }

    private void addMenuItems() {
        pluginWorkspace.addMenuBarCustomizer(new MenuBarCustomizer() {
            @Override
            public void customizeMainMenu(MenuBarCustomizer menuBar) {
                // Create AI menu
                Menu aiMenu = new Menu("AI Markup");
                
                // Add markup action
                JMenuItem markupItem = new JMenuItem(new AIMarkupAction(
                    "Markup Selected Text", pluginWorkspace, configManager));
                aiMenu.add(markupItem);
                
                // Add separator
                aiMenu.addSeparator();
                
                // Add configuration action
                JMenuItem configItem = new JMenuItem(new AIConfigurationAction(
                    "Configure AI Service", pluginWorkspace, configManager));
                aiMenu.add(configItem);
                
                // Add to menu bar
                menuBar.addMenu(aiMenu);
            }
        });
    }

    @Override
    public boolean applicationClosing() {
        // Save configuration on exit
        if (configManager != null) {
            configManager.saveConfiguration();
        }
        return true;
    }
}

/**
 * Action for AI text markup
 */
class AIMarkupAction extends AbstractAction {
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    public AIMarkupAction(String name, StandalonePluginWorkspace workspace, 
                         AIConfigurationManager configManager) {
        super(name);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        // Get selected text from current editor
        String selectedText = getSelectedText();
        
        if (selectedText == null || selectedText.trim().isEmpty()) {
            pluginWorkspace.showInformationMessage("Please select text to markup.");
            return;
        }

        // Check if configuration is valid
        if (!configManager.isConfigured()) {
            pluginWorkspace.showInformationMessage(
                "Please configure AI service first (AI Markup → Configure AI Service).");
            return;
        }

        // Show markup dialog
        AIMarkupDialog dialog = new AIMarkupDialog(
            pluginWorkspace, configManager, selectedText);
        dialog.setVisible(true);
    }

    private String getSelectedText() {
        try {
            WSEditor currentEditor = pluginWorkspace.getCurrentEditorAccess(
                StandalonePluginWorkspace.MAIN_EDITING_AREA);
            
            if (currentEditor != null) {
                AuthorAccess authorAccess = currentEditor.getAuthorAccess();
                if (authorAccess != null) {
                    AuthorDocumentController controller = 
                        authorAccess.getDocumentController();
                    
                    int selectionStart = authorAccess.getEditorAccess()
                        .getSelectionStart();
                    int selectionEnd = authorAccess.getEditorAccess()
                        .getSelectionEnd();
                    
                    if (selectionStart != selectionEnd) {
                        return controller.getChars(selectionStart, 
                            selectionEnd - selectionStart);
                    }
                }
            }
        } catch (Exception ex) {
            ex.printStackTrace();
        }
        return null;
    }
}

/**
 * Action for AI configuration
 */
class AIConfigurationAction extends AbstractAction {
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;

    public AIConfigurationAction(String name, StandalonePluginWorkspace workspace,
                               AIConfigurationManager configManager) {
        super(name);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
        AIConfigurationDialog dialog = new AIConfigurationDialog(
            pluginWorkspace, configManager);
        dialog.setVisible(true);
    }
}
</VSCode.Cell>
<VSCode.Cell language="java">
// Configuration Manager Class

package com.oxygenxml.ai.markup;

import java.util.List;
import java.util.ArrayList;
import java.io.IOException;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.net.URI;
import java.time.Duration;

import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;
import ro.sync.exml.workspace.api.options.WSOptionsStorage;

import com.google.gson.Gson;
import com.google.gson.JsonObject;
import com.google.gson.JsonArray;

/**
 * Manages AI service configuration and API communication
 */
public class AIConfigurationManager {
    
    private static final String CONFIG_KEY_ENDPOINT = "ai.markup.endpoint";
    private static final String CONFIG_KEY_API_KEY = "ai.markup.apikey";
    private static final String CONFIG_KEY_MODEL = "ai.markup.model";
    private static final String CONFIG_KEY_TIMEOUT = "ai.markup.timeout";
    
    private static final String DEFAULT_ENDPOINT = "http://127.0.0.1:5000/ai";
    private static final int DEFAULT_TIMEOUT = 120;
    
    private StandalonePluginWorkspace pluginWorkspace;
    private WSOptionsStorage optionsStorage;
    private HttpClient httpClient;
    private Gson gson;
    
    public AIConfigurationManager(StandalonePluginWorkspace workspace) {
        this.pluginWorkspace = workspace;
        this.optionsStorage = workspace.getOptionsStorage();
        this.httpClient = HttpClient.newBuilder()
            .connectTimeout(Duration.ofSeconds(10))
            .build();
        this.gson = new Gson();
        
        // Load existing configuration
        loadConfiguration();
    }
    
    // Configuration properties
    private String endpoint = DEFAULT_ENDPOINT;
    private String apiKey = "";
    private String selectedModel = "";
    private int timeout = DEFAULT_TIMEOUT;
    
    public void loadConfiguration() {
        endpoint = optionsStorage.getOption(CONFIG_KEY_ENDPOINT, DEFAULT_ENDPOINT);
        apiKey = optionsStorage.getOption(CONFIG_KEY_API_KEY, "");
        selectedModel = optionsStorage.getOption(CONFIG_KEY_MODEL, "");
        timeout = Integer.parseInt(optionsStorage.getOption(CONFIG_KEY_TIMEOUT, 
            String.valueOf(DEFAULT_TIMEOUT)));
    }
    
    public void saveConfiguration() {
        optionsStorage.setOption(CONFIG_KEY_ENDPOINT, endpoint);
        optionsStorage.setOption(CONFIG_KEY_API_KEY, apiKey);
        optionsStorage.setOption(CONFIG_KEY_MODEL, selectedModel);
        optionsStorage.setOption(CONFIG_KEY_TIMEOUT, String.valueOf(timeout));
    }
    
    public boolean isConfigured() {
        return !endpoint.trim().isEmpty() && !apiKey.trim().isEmpty();
    }
    
    // Getters and setters
    public String getEndpoint() { return endpoint; }
    public void setEndpoint(String endpoint) { this.endpoint = endpoint; }
    
    public String getApiKey() { return apiKey; }
    public void setApiKey(String apiKey) { this.apiKey = apiKey; }
    
    public String getSelectedModel() { return selectedModel; }
    public void setSelectedModel(String model) { this.selectedModel = model; }
    
    public int getTimeout() { return timeout; }
    public void setTimeout(int timeout) { this.timeout = timeout; }
    
    /**
     * Test API connection and get available models
     */
    public List<String> getAvailableModels() throws Exception {
        String modelsUrl = endpoint + "/models";
        
        HttpRequest request = HttpRequest.newBuilder()
            .uri(URI.create(modelsUrl))
            .header("Authorization", "Bearer " + apiKey)
            .header("Content-Type", "application/json")
            .timeout(Duration.ofSeconds(10))
            .GET()
            .build();
        
        HttpResponse<String> response = httpClient.send(request, 
            HttpResponse.BodyHandlers.ofString());
        
        if (response.statusCode() != 200) {
            throw new Exception("Failed to get models: " + response.body());
        }
        
        // Parse response
        JsonObject jsonResponse = gson.fromJson(response.body(), JsonObject.class);
        JsonArray modelsArray = jsonResponse.getAsJsonArray("data");
        
        List<String> models = new ArrayList<>();
        for (int i = 0; i < modelsArray.size(); i++) {
            JsonObject model = modelsArray.get(i).getAsJsonObject();
            models.add(model.get("id").getAsString());
        }
        
        return models;
    }
    
    /**
     * Test API connection
     */
    public boolean testConnection() {
        try {
            getAvailableModels();
            return true;
        } catch (Exception e) {
            return false;
        }
    }
    
    /**
     * Send markup request to AI service
     */
    public String requestMarkup(String text, String prompt) throws Exception {
        String completionsUrl = endpoint + "/chat/completions";
        
        // Prepare request body
        JsonObject requestBody = new JsonObject();
        requestBody.addProperty("model", selectedModel);
        requestBody.addProperty("max_tokens", 1500);
        requestBody.addProperty("temperature", 0.7);
        
        JsonArray messages = new JsonArray();
        
        // System message with prompt
        JsonObject systemMessage = new JsonObject();
        systemMessage.addProperty("role", "system");
        systemMessage.addProperty("content", prompt);
        messages.add(systemMessage);
        
        // User message with text to markup
        JsonObject userMessage = new JsonObject();
        userMessage.addProperty("role", "user");
        userMessage.addProperty("content", text);
        messages.add(userMessage);
        
        requestBody.add("messages", messages);
        
        HttpRequest request = HttpRequest.newBuilder()
            .uri(URI.create(completionsUrl))
            .header("Authorization", "Bearer " + apiKey)
            .header("Content-Type", "application/json")
            .timeout(Duration.ofSeconds(timeout))
            .POST(HttpRequest.BodyPublishers.ofString(gson.toJson(requestBody)))
            .build();
        
        HttpResponse<String> response = httpClient.send(request, 
            HttpResponse.BodyHandlers.ofString());
        
        if (response.statusCode() != 200) {
            throw new Exception("AI request failed: " + response.body());
        }
        
        // Parse response
        JsonObject jsonResponse = gson.fromJson(response.body(), JsonObject.class);
        JsonArray choices = jsonResponse.getAsJsonArray("choices");
        
        if (choices.size() > 0) {
            JsonObject firstChoice = choices.get(0).getAsJsonObject();
            JsonObject message = firstChoice.getAsJsonObject("message");
            return message.get("content").getAsString();
        }
        
        throw new Exception("No response from AI service");
    }
}
</VSCode.Cell>
<VSCode.Cell language="java">
// UI Dialog Classes

package com.oxygenxml.ai.markup;

import java.awt.*;
import java.awt.event.*;
import java.util.List;
import javax.swing.*;
import javax.swing.border.EmptyBorder;

import ro.sync.exml.workspace.api.standalone.StandalonePluginWorkspace;

/**
 * Configuration dialog for AI service settings
 */
public class AIConfigurationDialog extends JDialog {
    
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;
    
    private JTextField endpointField;
    private JPasswordField apiKeyField;
    private JComboBox<String> modelComboBox;
    private JSpinner timeoutSpinner;
    private JButton testButton;
    private JLabel statusLabel;
    
    public AIConfigurationDialog(StandalonePluginWorkspace workspace, 
                               AIConfigurationManager configManager) {
        super((Frame) workspace.getParentFrame(), "AI Service Configuration", true);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
        
        initComponents();
        loadSettings();
        setupEventHandlers();
        
        setDefaultCloseOperation(DISPOSE_ON_CLOSE);
        pack();
        setLocationRelativeTo(workspace.getParentFrame());
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        
        // Main panel
        JPanel mainPanel = new JPanel(new GridBagLayout());
        mainPanel.setBorder(new EmptyBorder(15, 15, 15, 15));
        
        GridBagConstraints gbc = new GridBagConstraints();
        gbc.insets = new Insets(5, 5, 5, 5);
        gbc.anchor = GridBagConstraints.WEST;
        
        // Endpoint
        gbc.gridx = 0; gbc.gridy = 0;
        mainPanel.add(new JLabel("API Endpoint:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        endpointField = new JTextField(30);
        mainPanel.add(endpointField, gbc);
        
        // API Key
        gbc.gridx = 0; gbc.gridy = 1; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("API Key:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        apiKeyField = new JPasswordField(30);
        mainPanel.add(apiKeyField, gbc);
        
        // Model
        gbc.gridx = 0; gbc.gridy = 2; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("Model:"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        modelComboBox = new JComboBox<>();
        modelComboBox.setEditable(true);
        mainPanel.add(modelComboBox, gbc);
        
        // Timeout
        gbc.gridx = 0; gbc.gridy = 3; gbc.weightx = 0; gbc.fill = GridBagConstraints.NONE;
        mainPanel.add(new JLabel("Timeout (seconds):"), gbc);
        
        gbc.gridx = 1; gbc.weightx = 1.0; gbc.fill = GridBagConstraints.HORIZONTAL;
        timeoutSpinner = new JSpinner(new SpinnerNumberModel(120, 10, 600, 10));
        mainPanel.add(timeoutSpinner, gbc);
        
        // Test button
        gbc.gridx = 0; gbc.gridy = 4; gbc.gridwidth = 2; gbc.anchor = GridBagConstraints.CENTER;
        testButton = new JButton("Test Connection & Load Models");
        mainPanel.add(testButton, gbc);
        
        // Status label
        gbc.gridx = 0; gbc.gridy = 5; gbc.gridwidth = 2;
        statusLabel = new JLabel(" ");
        statusLabel.setHorizontalAlignment(SwingConstants.CENTER);
        mainPanel.add(statusLabel, gbc);
        
        add(mainPanel, BorderLayout.CENTER);
        
        // Button panel
        JPanel buttonPanel = new JPanel(new FlowLayout());
        JButton okButton = new JButton("OK");
        JButton cancelButton = new JButton("Cancel");
        
        buttonPanel.add(okButton);
        buttonPanel.add(cancelButton);
        add(buttonPanel, BorderLayout.SOUTH);
        
        // Button event handlers
        okButton.addActionListener(e -> saveAndClose());
        cancelButton.addActionListener(e -> dispose());
    }
    
    private void loadSettings() {
        endpointField.setText(configManager.getEndpoint());
        apiKeyField.setText(configManager.getApiKey());
        timeoutSpinner.setValue(configManager.getTimeout());
        
        // Try to load models if configuration exists
        if (configManager.isConfigured()) {
            loadModels();
        }
    }
    
    private void setupEventHandlers() {
        testButton.addActionListener(e -> testConnectionAndLoadModels());
    }
    
    private void testConnectionAndLoadModels() {
        // Update config with current values
        configManager.setEndpoint(endpointField.getText().trim());
        configManager.setApiKey(new String(apiKeyField.getPassword()));
        configManager.setTimeout((Integer) timeoutSpinner.getValue());
        
        testButton.setEnabled(false);
        statusLabel.setText("Testing connection...");
        statusLabel.setForeground(Color.BLUE);
        
        // Run in background thread
        SwingWorker<Void, Void> worker = new SwingWorker<Void, Void>() {
            private List<String> models;
            private Exception error;
            
            @Override
            protected Void doInBackground() throws Exception {
                try {
                    models = configManager.getAvailableModels();
                } catch (Exception e) {
                    error = e;
                }
                return null;
            }
            
            @Override
            protected void done() {
                testButton.setEnabled(true);
                
                if (error == null && models != null) {
                    // Success
                    statusLabel.setText("✓ Connection successful! Models loaded.");
                    statusLabel.setForeground(Color.GREEN);
                    
                    // Update model combo box
                    modelComboBox.removeAllItems();
                    for (String model : models) {
                        modelComboBox.addItem(model);
                    }
                    
                    // Select previously selected model if available
                    String selectedModel = configManager.getSelectedModel();
                    if (!selectedModel.isEmpty() && models.contains(selectedModel)) {
                        modelComboBox.setSelectedItem(selectedModel);
                    }
                    
                } else {
                    // Error
                    statusLabel.setText("✗ Connection failed: " + 
                        (error != null ? error.getMessage() : "Unknown error"));
                    statusLabel.setForeground(Color.RED);
                }
            }
        };
        
        worker.execute();
    }
    
    private void loadModels() {
        SwingWorker<Void, Void> worker = new SwingWorker<Void, Void>() {
            private List<String> models;
            
            @Override
            protected Void doInBackground() throws Exception {
                try {
                    models = configManager.getAvailableModels();
                } catch (Exception e) {
                    // Ignore errors when loading initial models
                }
                return null;
            }
            
            @Override
            protected void done() {
                if (models != null) {
                    modelComboBox.removeAllItems();
                    for (String model : models) {
                        modelComboBox.addItem(model);
                    }
                    
                    String selectedModel = configManager.getSelectedModel();
                    if (!selectedModel.isEmpty() && models.contains(selectedModel)) {
                        modelComboBox.setSelectedItem(selectedModel);
                    }
                }
            }
        };
        
        worker.execute();
    }
    
    private void saveAndClose() {
        // Validate inputs
        String endpoint = endpointField.getText().trim();
        String apiKey = new String(apiKeyField.getPassword());
        
        if (endpoint.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter an API endpoint.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        if (apiKey.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter an API key.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        // Save configuration
        configManager.setEndpoint(endpoint);
        configManager.setApiKey(apiKey);
        configManager.setTimeout((Integer) timeoutSpinner.getValue());
        
        Object selectedModel = modelComboBox.getSelectedItem();
        if (selectedModel != null) {
            configManager.setSelectedModel(selectedModel.toString());
        }
        
        configManager.saveConfiguration();
        
        dispose();
    }
}

/**
 * Main markup dialog for text processing
 */
public class AIMarkupDialog extends JDialog {
    
    private StandalonePluginWorkspace pluginWorkspace;
    private AIConfigurationManager configManager;
    private String originalText;
    
    private JTextArea promptArea;
    private JTextArea originalTextArea;
    private JTextArea resultTextArea;
    private JButton processButton;
    private JButton replaceButton;
    private JButton copyButton;
    private JButton previewButton;
    private JLabel statusLabel;
    
    public AIMarkupDialog(StandalonePluginWorkspace workspace, 
                         AIConfigurationManager configManager, String selectedText) {
        super((Frame) workspace.getParentFrame(), "AI Text Markup", true);
        this.pluginWorkspace = workspace;
        this.configManager = configManager;
        this.originalText = selectedText;
        
        initComponents();
        setupEventHandlers();
        
        setDefaultCloseOperation(DISPOSE_ON_CLOSE);
        pack();
        setLocationRelativeTo(workspace.getParentFrame());
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        setSize(800, 600);
        
        // Create main panel with split panes
        JSplitPane mainSplit = new JSplitPane(JSplitPane.VERTICAL_SPLIT);
        
        // Top panel for prompt and controls
        JPanel topPanel = new JPanel(new BorderLayout());
        topPanel.setBorder(BorderFactory.createTitledBorder("AI Prompt"));
        
        promptArea = new JTextArea(4, 50);
        promptArea.setText("Please markup the following text with appropriate XML tags. " +
            "Follow the existing document structure and use standard markup patterns.");
        promptArea.setLineWrap(true);
        promptArea.setWrapStyleWord(true);
        topPanel.add(new JScrollPane(promptArea), BorderLayout.CENTER);
        
        // Control panel
        JPanel controlPanel = new JPanel(new FlowLayout());
        processButton = new JButton("Process with AI");
        controlPanel.add(processButton);
        
        statusLabel = new JLabel("Model: " + configManager.getSelectedModel());
        controlPanel.add(statusLabel);
        
        topPanel.add(controlPanel, BorderLayout.SOUTH);
        
        // Bottom panel for text comparison
        JSplitPane textSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        
        // Original text panel
        JPanel originalPanel = new JPanel(new BorderLayout());
        originalPanel.setBorder(BorderFactory.createTitledBorder("Original Text"));
        originalTextArea = new JTextArea(15, 30);
        originalTextArea.setText(originalText);
        originalTextArea.setEditable(false);
        originalTextArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        originalPanel.add(new JScrollPane(originalTextArea), BorderLayout.CENTER);
        
        // Result text panel
        JPanel resultPanel = new JPanel(new BorderLayout());
        resultPanel.setBorder(BorderFactory.createTitledBorder("AI Result"));
        resultTextArea = new JTextArea(15, 30);
        resultTextArea.setEditable(true);
        resultTextArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        resultPanel.add(new JScrollPane(resultTextArea), BorderLayout.CENTER);
        
        // Button panel for results
        JPanel resultButtonPanel = new JPanel(new FlowLayout());
        replaceButton = new JButton("Replace Original");
        copyButton = new JButton("Copy Result");
        previewButton = new JButton("Preview Diff");
        
        replaceButton.setEnabled(false);
        copyButton.setEnabled(false);
        previewButton.setEnabled(false);
        
        resultButtonPanel.add(replaceButton);
        resultButtonPanel.add(copyButton);
        resultButtonPanel.add(previewButton);
        resultPanel.add(resultButtonPanel, BorderLayout.SOUTH);
        
        textSplit.setLeftComponent(originalPanel);
        textSplit.setRightComponent(resultPanel);
        textSplit.setDividerLocation(400);
        
        mainSplit.setTopComponent(topPanel);
        mainSplit.setBottomComponent(textSplit);
        mainSplit.setDividerLocation(150);
        
        add(mainSplit, BorderLayout.CENTER);
        
        // Bottom button panel
        JPanel bottomButtonPanel = new JPanel(new FlowLayout());
        JButton closeButton = new JButton("Close");
        closeButton.addActionListener(e -> dispose());
        bottomButtonPanel.add(closeButton);
        
        add(bottomButtonPanel, BorderLayout.SOUTH);
    }
    
    private void setupEventHandlers() {
        processButton.addActionListener(e -> processWithAI());
        replaceButton.addActionListener(e -> replaceText());
        copyButton.addActionListener(e -> copyResult());
        previewButton.addActionListener(e -> showPreview());
    }
    
    private void processWithAI() {
        String prompt = promptArea.getText().trim();
        if (prompt.isEmpty()) {
            JOptionPane.showMessageDialog(this, "Please enter a prompt.", 
                "Validation Error", JOptionPane.ERROR_MESSAGE);
            return;
        }
        
        processButton.setEnabled(false);
        statusLabel.setText("Processing with AI...");
        statusLabel.setForeground(Color.BLUE);
        resultTextArea.setText("");
        
        SwingWorker<String, Void> worker = new SwingWorker<String, Void>() {
            private Exception error;
            
            @Override
            protected String doInBackground() throws Exception {
                try {
                    return configManager.requestMarkup(originalText, prompt);
                } catch (Exception e) {
                    error = e;
                    return null;
                }
            }
            
            @Override
            protected void done() {
                processButton.setEnabled(true);
                
                try {
                    String result = get();
                    if (result != null) {
                        resultTextArea.setText(result);
                        statusLabel.setText("✓ Processing completed");
                        statusLabel.setForeground(Color.GREEN);
                        
                        replaceButton.setEnabled(true);
                        copyButton.setEnabled(true);
                        previewButton.setEnabled(true);
                    } else {
                        statusLabel.setText("✗ Processing failed: " + 
                            (error != null ? error.getMessage() : "Unknown error"));
                        statusLabel.setForeground(Color.RED);
                    }
                } catch (Exception e) {
                    statusLabel.setText("✗ Processing failed: " + e.getMessage());
                    statusLabel.setForeground(Color.RED);
                }
            }
        };
        
        worker.execute();
    }
    
    private void replaceText() {
        String result = resultTextArea.getText();
        if (result.trim().isEmpty()) {
            return;
        }
        
        int choice = JOptionPane.showConfirmDialog(this, 
            "Replace the selected text in the document with the AI result?", 
            "Confirm Replacement", JOptionPane.YES_NO_OPTION);
        
        if (choice == JOptionPane.YES_OPTION) {
            try {
                replaceSelectedText(result);
                JOptionPane.showMessageDialog(this, "Text replaced successfully!");
                dispose();
            } catch (Exception e) {
                JOptionPane.showMessageDialog(this, 
                    "Error replacing text: " + e.getMessage(), 
                    "Error", JOptionPane.ERROR_MESSAGE);
            }
        }
    }
    
    private void copyResult() {
        String result = resultTextArea.getText();
        if (!result.trim().isEmpty()) {
            Toolkit.getDefaultToolkit().getSystemClipboard()
                .setContents(new StringSelection(result), null);
            JOptionPane.showMessageDialog(this, "Result copied to clipboard!");
        }
    }
    
    private void showPreview() {
        String original = originalTextArea.getText();
        String result = resultTextArea.getText();
        
        if (result.trim().isEmpty()) {
            return;
        }
        
        // Create preview dialog
        JDialog previewDialog = new JDialog(this, "Preview Changes", true);
        previewDialog.setLayout(new BorderLayout());
        previewDialog.setSize(900, 500);
        
        // Create diff view (simple side-by-side comparison)
        JSplitPane diffSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        
        JPanel beforePanel = new JPanel(new BorderLayout());
        beforePanel.setBorder(BorderFactory.createTitledBorder("Before (Original)"));
        JTextArea beforeArea = new JTextArea(original);
        beforeArea.setEditable(false);
        beforeArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 11));
        beforePanel.add(new JScrollPane(beforeArea), BorderLayout.CENTER);
        
        JPanel afterPanel = new JPanel(new BorderLayout());
        afterPanel.setBorder(BorderFactory.createTitledBorder("After (AI Result)"));
        JTextArea afterArea = new JTextArea(result);
        afterArea.setEditable(false);
        afterArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 11));
        afterPanel.add(new JScrollPane(afterArea), BorderLayout.CENTER);
        
        diffSplit.setLeftComponent(beforePanel);
        diffSplit.setRightComponent(afterPanel);
        diffSplit.setDividerLocation(450);
        
        previewDialog.add(diffSplit, BorderLayout.CENTER);
        
        JPanel previewButtonPanel = new JPanel(new FlowLayout());
        JButton acceptButton = new JButton("Accept & Replace");
        JButton cancelButton = new JButton("Cancel");
        
        acceptButton.addActionListener(e -> {
            previewDialog.dispose();
            replaceText();
        });
        cancelButton.addActionListener(e -> previewDialog.dispose());
        
        previewButtonPanel.add(acceptButton);
        previewButtonPanel.add(cancelButton);
        previewDialog.add(previewButtonPanel, BorderLayout.SOUTH);
        
        previewDialog.setLocationRelativeTo(this);
        previewDialog.setVisible(true);
    }
    
    private void replaceSelectedText(String replacement) throws Exception {
        WSEditor currentEditor = pluginWorkspace.getCurrentEditorAccess(
            StandalonePluginWorkspace.MAIN_EDITING_AREA);
        
        if (currentEditor != null) {
            AuthorAccess authorAccess = currentEditor.getAuthorAccess();
            if (authorAccess != null) {
                AuthorDocumentController controller = 
                    authorAccess.getDocumentController();
                
                int selectionStart = authorAccess.getEditorAccess().getSelectionStart();
                int selectionEnd = authorAccess.getEditorAccess().getSelectionEnd();
                
                if (selectionStart != selectionEnd) {
                    controller.delete(selectionStart, selectionEnd - selectionStart);
                    controller.insertText(selectionStart, replacement);
                }
            }
        }
    }
}
</VSCode.Cell>
<VSCode.Cell language="xml">
<!-- filepath: c:\Project\OCR\Oxygen-add-on.ipynb -->
<!-- Plugin Configuration Files -->

<!-- plugin.xml - Main plugin descriptor -->
<?xml version="1.0" encoding="UTF-8"?>
<plugin
    name="AI Text Markup"
    description="AI-powered text markup using local Ollama models via Flask API"
    version="1.0.0"
    vendor="Custom Development"
    class="com.oxygenxml.ai.markup.AITextMarkupPlugin">
    
    <runtime>
        <library name="lib/ai-markup-plugin.jar"/>
        <library name="lib/gson-2.8.9.jar"/>
    </runtime>
    
    <extension point="WorkspaceAccess" class="com.oxygenxml.ai.markup.AITextMarkupPlugin"/>
</plugin>

<!-- addon.xml - Add-on descriptor for packaging -->
<?xml version="1.0" encoding="UTF-8"?>
<xt:extensions xmlns:xt="http://www.oxygenxml.com/ns/extension"
              xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
              xsi:schemaLocation="http://www.oxygenxml.com/ns/extension 
                                 http://www.oxygenxml.com/ns/extension/extensions.xsd">
    
    <xt:extension id="ai.text.markup">
        <xt:location href="ai-markup-plugin.jar"/>
        <xt:version>1.0.0</xt:version>
        <xt:oxy_version>20.0+</xt:oxy_version>
        <xt:type>plugin</xt:type>
        <xt:author>Custom Development</xt:author>
        <xt:name>AI Text Markup</xt:name>
        <xt:description>
            AI-powered text markup using local Ollama models via Flask API.
            Select text in XML documents and apply intelligent markup using local AI models.
        </xt:description>
        <xt:license>
            <![CDATA[
            Custom License - For personal and educational use.
            ]]>
        </xt:license>
    </xt:extension>
</xt:extensions>

<!-- build.xml - Ant build script for compilation -->
<?xml version="1.0" encoding="UTF-8"?>
<project name="ai-markup-plugin" default="jar" basedir=".">
    
    <property name="src.dir" value="src"/>
    <property name="build.dir" value="build"/>
    <property name="lib.dir" value="lib"/>
    <property name="oxygen.dir" value="C:/Program Files/Oxygen XML Author 27"/>
    
    <path id="classpath">
        <fileset dir="${oxygen.dir}/lib" includes="*.jar"/>
        <fileset dir="${lib.dir}" includes="*.jar"/>
    </path>
    
    <target name="init">
        <mkdir dir="${build.dir}"/>
    </target>
    
    <target name="compile" depends="init">
        <javac srcdir="${src.dir}" 
               destdir="${build.dir}" 
               classpathref="classpath"
               includeantruntime="false"/>
    </target>
    
    <target name="jar" depends="compile">
        <jar destfile="${lib.dir}/ai-markup-plugin.jar" basedir="${build.dir}">
            <manifest>
                <attribute name="Plugin-Class" value="com.oxygenxml.ai.markup.AITextMarkupPlugin"/>
                <attribute name="Plugin-Name" value="AI Text Markup"/>
                <attribute name="Plugin-Version" value="1.0.0"/>
            </manifest>
        </jar>
    </target>
    
    <target name="clean">
        <delete dir="${build.dir}"/>
    </target>
    
    <target name="addon" depends="jar">
        <zip destfile="ai-markup-addon.zip">
            <fileset dir="." includes="plugin.xml,addon.xml"/>
            <fileset dir="${lib.dir}" includes="*.jar"/>
        </zip>
    </target>
</project>
</VSCode.Cell>
<VSCode.Cell language="markdown">
## Installation Instructions

### 1. Prerequisites
- Download required dependencies:
  - Gson library: `gson-2.8.9.jar`
  - Place in `lib/` directory

### 2. Build the Plugin
```bash
# Compile and create JAR
ant jar

# Create add-on package
ant addon
```

### 3. Install in Oxygen XML Author
1. Open Oxygen XML Author
2. Go to **Help → Install new add-ons**
3. Click **Browse** and select `ai-markup-addon.zip`
4. Follow installation wizard
5. Restart Oxygen XML Author

### 4. Configure AI Service
1. Go to **AI Markup → Configure AI Service**
2. Enter your Flask API details:
   - **API Endpoint**: `http://127.0.0.1:5000/ai`
   - **API Key**: From your Flask API key manager
   - **Timeout**: 120 seconds (adjust as needed)
3. Click **Test Connection & Load Models**
4. Select desired model from dropdown
5. Click **OK** to save

### 5. Usage
1. Open an XML document in Oxygen
2. Select text you want to markup
3. Go to **AI Markup → Markup Selected Text**
4. Enter or modify the AI prompt
5. Click **Process with AI**
6. Review results and choose:
   - **Replace Original**: Replace selected text in document
   - **Copy Result**: Copy to clipboard
   - **Preview Diff**: Side-by-side comparison before replacing

## Features
- ✅ Integration with local Flask API
- ✅ Model selection from Ollama
- ✅ Configurable prompts
- ✅ Preview before replacement
- ✅ Side-by-side comparison
- ✅ Copy to clipboard functionality
- ✅ Persistent configuration
- ✅ Error handling and validation

## Configuration Files
- Settings stored in Oxygen's options storage
- Automatically persisted between sessions
- Test connection validates API accessibility
- Dynamic model loading from API
</VSCode.Cell>





Made changes.

### [Add-on-UI-preview.ipynb](file:///c%3A/Project/OCR/Add-on-UI-preview.ipynb)

Create preview and comparison UI components.



In [ ]:
<VSCode.Cell language="markdown">
# Add-on UI Preview Components

Advanced preview and comparison functionality for the AI Text Markup add-on.
</VSCode.Cell>
<VSCode.Cell language="java">
// filepath: c:\Project\OCR\Add-on-UI-preview.ipynb
// Enhanced Preview Dialog with Diff Highlighting

package com.oxygenxml.ai.markup.preview;

import java.awt.*;
import java.awt.event.*;
import java.util.*;
import java.util.List;
import javax.swing.*;
import javax.swing.text.*;

/**
 * Advanced preview dialog with diff highlighting
 */
public class DiffPreviewDialog extends JDialog {
    
    private String originalText;
    private String modifiedText;
    private JTextPane beforePane;
    private JTextPane afterPane;
    
    public DiffPreviewDialog(Frame parent, String original, String modified) {
        super(parent, "Preview Changes - Diff View", true);
        this.originalText = original;
        this.modifiedText = modified;
        
        initComponents();
        highlightDifferences();
        
        setDefaultCloseOperation(DISPOSE_ON_CLOSE);
        setSize(1000, 600);
        setLocationRelativeTo(parent);
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        
        // Header panel
        JPanel headerPanel = new JPanel(new BorderLayout());
        headerPanel.setBorder(BorderFactory.createEmptyBorder(10, 10, 5, 10));
        
        JLabel titleLabel = new JLabel("Preview Changes");
        titleLabel.setFont(titleLabel.getFont().deriveFont(Font.BOLD, 16f));
        headerPanel.add(titleLabel, BorderLayout.WEST);
        
        // Legend
        JPanel legendPanel = new JPanel(new FlowLayout());
        legendPanel.add(createLegendItem("Removed", Color.RED.brighter()));
        legendPanel.add(createLegendItem("Added", Color.GREEN.brighter()));
        legendPanel.add(createLegendItem("Modified", Color.YELLOW.brighter()));
        headerPanel.add(legendPanel, BorderLayout.EAST);
        
        add(headerPanel, BorderLayout.NORTH);
        
        // Main content - split pane
        JSplitPane splitPane = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        splitPane.setBorder(BorderFactory.createEmptyBorder(5, 10, 10, 10));
        
        // Before panel
        JPanel beforePanel = new JPanel(new BorderLayout());
        beforePanel.setBorder(BorderFactory.createTitledBorder("Before (Original)"));
        
        beforePane = new JTextPane();
        beforePane.setEditable(false);
        beforePane.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        beforePane.setText(originalText);
        
        JScrollPane beforeScroll = new JScrollPane(beforePane);
        beforeScroll.setPreferredSize(new Dimension(450, 400));
        beforePanel.add(beforeScroll, BorderLayout.CENTER);
        
        // After panel
        JPanel afterPanel = new JPanel(new BorderLayout());
        afterPanel.setBorder(BorderFactory.createTitledBorder("After (AI Result)"));
        
        afterPane = new JTextPane();
        afterPane.setEditable(false);
        afterPane.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        afterPane.setText(modifiedText);
        
        JScrollPane afterScroll = new JScrollPane(afterPane);
        afterScroll.setPreferredSize(new Dimension(450, 400));
        afterPanel.add(afterScroll, BorderLayout.CENTER);
        
        splitPane.setLeftComponent(beforePanel);
        splitPane.setRightComponent(afterPanel);
        splitPane.setDividerLocation(500);
        
        add(splitPane, BorderLayout.CENTER);
        
        // Button panel
        JPanel buttonPanel = new JPanel(new FlowLayout());
        
        JButton acceptButton = new JButton("Accept Changes");
        acceptButton.setBackground(Color.GREEN.darker());
        acceptButton.setForeground(Color.WHITE);
        acceptButton.setPreferredSize(new Dimension(150, 30));
        
        JButton rejectButton = new JButton("Reject Changes");
        rejectButton.setBackground(Color.RED.darker());
        rejectButton.setForeground(Color.WHITE);
        rejectButton.setPreferredSize(new Dimension(150, 30));
        
        JButton exportButton = new JButton("Export Diff");
        exportButton.setPreferredSize(new Dimension(120, 30));
        
        buttonPanel.add(acceptButton);
        buttonPanel.add(rejectButton);
        buttonPanel.add(exportButton);
        
        add(buttonPanel, BorderLayout.SOUTH);
        
        // Button actions
        acceptButton.addActionListener(e -> {
            setDialogResult(DialogResult.ACCEPT);
            dispose();
        });
        
        rejectButton.addActionListener(e -> {
            setDialogResult(DialogResult.REJECT);
            dispose();
        });
        
        exportButton.addActionListener(e -> exportDiff());
    }
    
    private JLabel createLegendItem(String text, Color color) {
        JLabel label = new JLabel("■ " + text);
        label.setForeground(color);
        label.setFont(label.getFont().deriveFont(Font.BOLD));
        return label;
    }
    
    private void highlightDifferences() {
        // Simple word-based diff highlighting
        List<DiffChunk> diffs = calculateDifferences();
        
        // Apply highlighting to before pane
        highlightText(beforePane, originalText, diffs, true);
        
        // Apply highlighting to after pane  
        highlightText(afterPane, modifiedText, diffs, false);
    }
    
    private List<DiffChunk> calculateDifferences() {
        // Simple diff implementation
        String[] originalWords = originalText.split("\\s+");
        String[] modifiedWords = modifiedText.split("\\s+");
        
        List<DiffChunk> chunks = new ArrayList<>();
        
        // Use simple LCS algorithm for basic diff
        int[][] lcs = computeLCS(originalWords, modifiedWords);
        traceback(lcs, originalWords, modifiedWords, chunks);
        
        return chunks;
    }
    
    private int[][] computeLCS(String[] a, String[] b) {
        int m = a.length;
        int n = b.length;
        int[][] dp = new int[m + 1][n + 1];
        
        for (int i = 1; i <= m; i++) {
            for (int j = 1; j <= n; j++) {
                if (a[i-1].equals(b[j-1])) {
                    dp[i][j] = dp[i-1][j-1] + 1;
                } else {
                    dp[i][j] = Math.max(dp[i-1][j], dp[i][j-1]);
                }
            }
        }
        
        return dp;
    }
    
    private void traceback(int[][] lcs, String[] original, String[] modified, 
                          List<DiffChunk> chunks) {
        int i = original.length;
        int j = modified.length;
        
        while (i > 0 || j > 0) {
            if (i > 0 && j > 0 && original[i-1].equals(modified[j-1])) {
                chunks.add(0, new DiffChunk(DiffType.EQUAL, original[i-1], i-1, j-1));
                i--;
                j--;
            } else if (j > 0 && (i == 0 || lcs[i][j-1] >= lcs[i-1][j])) {
                chunks.add(0, new DiffChunk(DiffType.INSERT, modified[j-1], -1, j-1));
                j--;
            } else if (i > 0 && (j == 0 || lcs[i][j-1] < lcs[i-1][j])) {
                chunks.add(0, new DiffChunk(DiffType.DELETE, original[i-1], i-1, -1));
                i--;
            }
        }
    }
    
    private void highlightText(JTextPane pane, String text, List<DiffChunk> diffs, boolean isOriginal) {
        StyledDocument doc = pane.getStyledDocument();
        
        // Clear existing styles
        doc.setCharacterAttributes(0, doc.getLength(), 
            pane.getStyle(StyleContext.DEFAULT_STYLE), true);
        
        // Define styles
        Style deletedStyle = pane.addStyle("deleted", null);
        StyleConstants.setBackground(deletedStyle, new Color(255, 200, 200));
        
        Style insertedStyle = pane.addStyle("inserted", null);
        StyleConstants.setBackground(insertedStyle, new Color(200, 255, 200));
        
        Style modifiedStyle = pane.addStyle("modified", null);
        StyleConstants.setBackground(modifiedStyle, new Color(255, 255, 200));
        
        // Apply highlights based on diff chunks
        int position = 0;
        for (DiffChunk chunk : diffs) {
            String word = chunk.text;
            int wordStart = text.indexOf(word, position);
            
            if (wordStart >= 0) {
                int wordEnd = wordStart + word.length();
                
                Style style = null;
                if (chunk.type == DiffType.DELETE && isOriginal) {
                    style = deletedStyle;
                } else if (chunk.type == DiffType.INSERT && !isOriginal) {
                    style = insertedStyle;
                }
                
                if (style != null) {
                    doc.setCharacterAttributes(wordStart, word.length(), style, false);
                }
                
                position = wordEnd;
            }
        }
    }
    
    private void exportDiff() {
        JFileChooser fileChooser = new JFileChooser();
        fileChooser.setDialogTitle("Export Diff Report");
        fileChooser.setSelectedFile(new java.io.File("diff-report.html"));
        
        if (fileChooser.showSaveDialog(this) == JFileChooser.APPROVE_OPTION) {
            try {
                java.io.File file = fileChooser.getSelectedFile();
                exportDiffToHTML(file);
                JOptionPane.showMessageDialog(this, "Diff exported successfully!");
            } catch (Exception e) {
                JOptionPane.showMessageDialog(this, "Error exporting diff: " + e.getMessage(),
                    "Export Error", JOptionPane.ERROR_MESSAGE);
            }
        }
    }
    
    private void exportDiffToHTML(java.io.File file) throws Exception {
        StringBuilder html = new StringBuilder();
        html.append("<!DOCTYPE html>\n<html>\n<head>\n");
        html.append("<title>Diff Report</title>\n");
        html.append("<style>\n");
        html.append(".deleted { background-color: #ffcccc; }\n");
        html.append(".inserted { background-color: #ccffcc; }\n");
        html.append(".modified { background-color: #ffffcc; }\n");
        html.append("pre { font-family: monospace; white-space: pre-wrap; }\n");
        html.append("</style>\n</head>\n<body>\n");
        
        html.append("<h1>AI Text Markup - Diff Report</h1>\n");
        html.append("<p>Generated: ").append(new java.util.Date()).append("</p>\n");
        
        html.append("<h2>Before (Original)</h2>\n");
        html.append("<pre>").append(escapeHtml(originalText)).append("</pre>\n");
        
        html.append("<h2>After (AI Result)</h2>\n");
        html.append("<pre>").append(escapeHtml(modifiedText)).append("</pre>\n");
        
        html.append("</body>\n</html>");
        
        try (java.io.FileWriter writer = new java.io.FileWriter(file)) {
            writer.write(html.toString());
        }
    }
    
    private String escapeHtml(String text) {
        return text.replace("&", "&amp;")
                  .replace("<", "&lt;")
                  .replace(">", "&gt;")
                  .replace("\"", "&quot;")
                  .replace("'", "&#39;");
    }
    
    // Dialog result handling
    public enum DialogResult {
        ACCEPT, REJECT, CANCEL
    }
    
    private DialogResult dialogResult = DialogResult.CANCEL;
    
    public DialogResult getDialogResult() {
        return dialogResult;
    }
    
    private void setDialogResult(DialogResult result) {
        this.dialogResult = result;
    }
    
    // Diff data structures
    enum DiffType {
        EQUAL, INSERT, DELETE
    }
    
    static class DiffChunk {
        DiffType type;
        String text;
        int originalIndex;
        int modifiedIndex;
        
        DiffChunk(DiffType type, String text, int originalIndex, int modifiedIndex) {
            this.type = type;
            this.text = text;
            this.originalIndex = originalIndex;
            this.modifiedIndex = modifiedIndex;
        }
    }
}
</VSCode.Cell>
<VSCode.Cell language="java">
// Real-time Preview Component

package com.oxygenxml.ai.markup.preview;

import java.awt.*;
import java.awt.event.*;
import javax.swing.*;
import javax.swing.event.DocumentEvent;
import javax.swing.event.DocumentListener;
import javax.swing.text.*;

/**
 * Real-time preview component that updates as user types
 */
public class RealTimePreviewPanel extends JPanel {
    
    private JTextArea inputArea;
    private JEditorPane previewPane;
    private Timer updateTimer;
    private XMLValidator validator;
    private JLabel statusLabel;
    
    public RealTimePreviewPanel() {
        initComponents();
        setupEventHandlers();
        this.validator = new XMLValidator();
    }
    
    private void initComponents() {
        setLayout(new BorderLayout());
        setBorder(BorderFactory.createTitledBorder("Real-time Preview"));
        
        // Create split pane
        JSplitPane splitPane = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT);
        
        // Input panel
        JPanel inputPanel = new JPanel(new BorderLayout());
        inputPanel.setBorder(BorderFactory.createTitledBorder("Editable Result"));
        
        inputArea = new JTextArea();
        inputArea.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 12));
        inputArea.setLineWrap(false);
        inputArea.setTabSize(2);
        
        // Add line numbers
        JScrollPane inputScroll = new JScrollPane(inputArea);
        inputScroll.setRowHeaderView(new LineNumberComponent(inputArea));
        inputPanel.add(inputScroll, BorderLayout.CENTER);
        
        // Preview panel
        JPanel previewPanel = new JPanel(new BorderLayout());
        previewPanel.setBorder(BorderFactory.createTitledBorder("XML Preview"));
        
        previewPane = new JEditorPane();
        previewPane.setContentType("text/html");
        previewPane.setEditable(false);
        previewPane.setFont(new Font(Font.MONOSPACED, Font.PLAIN, 11));
        
        JScrollPane previewScroll = new JScrollPane(previewPane);
        previewPanel.add(previewScroll, BorderLayout.CENTER);
        
        // Status panel
        statusLabel = new JLabel("Ready");
        statusLabel.setBorder(BorderFactory.createEmptyBorder(5, 5, 5, 5));
        previewPanel.add(statusLabel, BorderLayout.SOUTH);
        
        splitPane.setLeftComponent(inputPanel);
        splitPane.setRightComponent(previewPanel);
        splitPane.setDividerLocation(400);
        
        add(splitPane, BorderLayout.CENTER);
        
        // Toolbar
        JToolBar toolbar = new JToolBar();
        toolbar.setFloatable(false);
        
        JButton formatButton = new JButton("Format XML");
        formatButton.addActionListener(e -> formatXML());
        toolbar.add(formatButton);
        
        JButton validateButton = new JButton("Validate");
        validateButton.addActionListener(e -> validateXML());
        toolbar.add(validateButton);
        
        toolbar.addSeparator();
        
        JButton copyButton = new JButton("Copy Formatted");
        copyButton.addActionListener(e -> copyFormatted());
        toolbar.add(copyButton);
        
        add(toolbar, BorderLayout.NORTH);
    }
    
    private void setupEventHandlers() {
        // Update timer for real-time preview
        updateTimer = new Timer(500, e -> updatePreview());
        updateTimer.setRepeats(false);
        
        // Document listener for input changes
        inputArea.getDocument().addDocumentListener(new DocumentListener() {
            @Override
            public void insertUpdate(DocumentEvent e) { scheduleUpdate(); }
            
            @Override
            public void removeUpdate(DocumentEvent e) { scheduleUpdate(); }
            
            @Override
            public void changedUpdate(DocumentEvent e) { scheduleUpdate(); }
            
            private void scheduleUpdate() {
                updateTimer.restart();
            }
        });
    }
    
    private void updatePreview() {
        String xml = inputArea.getText();
        
        if (xml.trim().isEmpty()) {
            previewPane.setText("<html><body><i>Enter XML content to see preview</i></body></html>");
            statusLabel.setText("Ready");
            statusLabel.setForeground(Color.BLACK);
            return;
        }
        
        // Format and validate XML
        try {
            String formattedXML = XMLFormatter.format(xml);
            ValidationResult result = validator.validate(xml);
            
            // Create HTML preview
            String htmlPreview = createHTMLPreview(formattedXML, result);
            previewPane.setText(htmlPreview);
            
            // Update status
            if (result.isValid()) {
                statusLabel.setText("✓ Valid XML");
                statusLabel.setForeground(Color.GREEN.darker());
            } else {
                statusLabel.setText("⚠ XML Issues: " + result.getErrorCount());
                statusLabel.setForeground(Color.RED.darker());
            }
            
        } catch (Exception e) {
            previewPane.setText("<html><body><font color='red'>Error: " + 
                escapeHtml(e.getMessage()) + "</font></body></html>");
            statusLabel.setText("✗ Invalid XML");
            statusLabel.setForeground(Color.RED);
        }
    }
    
    private String createHTMLPreview(String xml, ValidationResult validation) {
        StringBuilder html = new StringBuilder();
        html.append("<html><head>");
        html.append("<style>");
        html.append("body { font-family: monospace; font-size: 11px; margin: 5px; }");
        html.append(".xml-element { color: blue; font-weight: bold; }");
        html.append(".xml-attribute { color: red; }");
        html.append(".xml-text { color: black; }");
        html.append(".xml-comment { color: green; font-style: italic; }");
        html.append(".error { background-color: #ffeeee; border-left: 3px solid red; padding: 2px; margin: 2px 0; }");
        html.append(".warning { background-color: #fffaee; border-left: 3px solid orange; padding: 2px; margin: 2px 0; }");
        html.append("</style>");
        html.append("</head><body>");
        
        // Add validation messages
        if (!validation.isValid()) {
            html.append("<div class='error'>");
            html.append("<strong>Validation Errors:</strong><br>");
            for (String error : validation.getErrors()) {
                html.append("• ").append(escapeHtml(error)).append("<br>");
            }
            html.append("</div>");
        }
        
        if (!validation.getWarnings().isEmpty()) {
            html.append("<div class='warning'>");
            html.append("<strong>Warnings:</strong><br>");
            for (String warning : validation.getWarnings()) {
                html.append("• ").append(escapeHtml(warning)).append("<br>");
            }
            html.append("</div>");
        }
        
        // Syntax highlight XML
        String highlightedXML = syntaxHighlightXML(xml);
        html.append("<pre>").append(highlightedXML).append("</pre>");
        
        html.append("</body></html>");
        return html.toString();
    }
    
    private String syntaxHighlightXML(String xml) {
        // Simple syntax highlighting
        return xml.replaceAll("(&lt;/?

Similar code found with 4 license types